In [1]:

import pickle
import os
import h5py
from PIL import Image
import numpy as np
import math
import random
import transforms3d
import tensorflow as tf

2024-10-31 01:59:45.121300: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-31 01:59:45.139288: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-31 01:59:45.144741: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-31 01:59:45.158665: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-31 01:59:46.521728: W tensorflow/comp

In [2]:
input_dir = r"/work/nselva2s/rnd/robile_data_corr1"
use_images = True

In [3]:
# Load the dataset

def load_data(input_dir, load_images=False):
    # Define paths to your pickle files
    image_file = os.path.join(input_dir, 'images.h5')
    laser_file = os.path.join(input_dir, 'lasers.pkl')
    odom_file = os.path.join(input_dir, 'odoms.pkl')
    goal_odom_file = os.path.join(input_dir, 'goal_odoms.pkl')
    velocity_file = os.path.join(input_dir, 'velocities.pkl')
    tf_file = os.path.join(input_dir, 'tfs.pkl')
    
    # Load the pickle files
    with open(laser_file, 'rb') as f:
        lasers_raw = pickle.load(f)
        lasers = []
        for laser in lasers_raw:
            lasers.append([100.0 if math.isnan(x) or math.isinf(x) else x for x in laser])
    with open(odom_file, 'rb') as f:
        current_poses = pickle.load(f)
    with open(goal_odom_file, 'rb') as f:
        goal_poses = pickle.load(f)
    with open(velocity_file, 'rb') as f:
        velocities = pickle.load(f)
    # Split velocities into linear and angular components
    linear_velocities = np.array(velocities)[:, :2]  # Linear velocity (x, y)
    angular_velocities = np.array(velocities)[:, 2]  # Angular velocity (z)
    with open(tf_file, 'rb') as f:
        tfs = pickle.load(f)
    if load_images:
        print('loading images')
        with h5py.File(image_file, 'r') as f:
            image_keys = list(f.keys())
            images = [Image.fromarray(f[key][:]) for key in image_keys]
        print('loaded images')
    else:
        images = [None]*len(lasers)

    combined_data = list(zip(images, lasers, current_poses, goal_poses, linear_velocities, angular_velocities, tfs))
    return  combined_data



data = load_data(os.path.join(input_dir, 'downsampled_data'), load_images=use_images)
print('loaded data')

loading images
loaded images
loaded data


In [4]:
import transforms3d

# Function to invert a transformation (translation + rotation)
def invert_transform(translation, rotation):
    # Convert the quaternion to a rotation matrix (3x3)
    rotation_matrix = transforms3d.quaternions.quat2mat([rotation[3], rotation[0], rotation[1], rotation[2]])[:3, :3]
    # Invert the rotation matrix (transpose of a rotation matrix is its inverse)
    rotation_matrix_inv = np.transpose(rotation_matrix)
    
    # Invert the translation (apply the inverse rotation to the negative translation)
    translation_inv = -np.dot(rotation_matrix_inv, [translation[0], translation[1], translation[2]])
    
    # Create the inverse quaternion (negate the vector part, keep the scalar part the same)
    rotation_inv = transforms3d.quaternions.qinverse([rotation[3], rotation[0], rotation[1], rotation[2]])
    return translation_inv, rotation_inv

# Function to apply the transformation (translation + rotation) to a point
def transform_point(translation, rotation, point):
    # Convert the point to a homogeneous vector (x, y, z, 1)
    point_homogeneous = np.array([point[0], point[1], point[2], 1.0])
    
    # Create the translation matrix (4x4)
    translation_matrix = np.identity(4)
    translation_matrix[0, 3] = translation[0]
    translation_matrix[1, 3] = translation[1]
    translation_matrix[2, 3] = translation[2]
    # Create the transformation matrix from translation and rotation
    rotation_matrix = np.identity(4)
    rotation_matrix[:3, :3] = transforms3d.quaternions.quat2mat(rotation)[:3, :3]
    # Combine translation and rotation into a single transformation matrix
    transformation_matrix = np.dot(translation_matrix, rotation_matrix)
    # Apply the transformation to the point
    transformed_point = np.dot(transformation_matrix, point_homogeneous)
    
    
    # Return the transformed point (x, y, z)
    return transformed_point[:3]

# Function to transform a point from odom to base_link frame
def transform_pose_to_base_link(translation, rotation, point_in_odom):
    # Get the inverse of the transformation
    translation_inv, rotation_inv = invert_transform(translation, rotation)
    
    # Convert the point from odom to base_link using the inverted transformation
    point_in_base_link = transform_point(translation_inv, rotation_inv, point_in_odom)
    
    return point_in_base_link

In [5]:
lasers = []
current_poses = []
goal_poses = []
linear_velocities = []
angular_velocities = []
tfs = []
images = []
goals = []
target_distance = 0.2
forward_goal_thresh = 1.4
sampling_probability = 0.4

for i, (_, _, _, goal_pose1, _, _, _) in enumerate(data):
    for j, (image2, laser2, current_pose2, _, linear_velocity2, angular_velocity2, tf2) in enumerate(data[i:]):   
        if random.random() > sampling_probability:
            continue
        distance = np.sqrt((goal_pose1[0] - current_pose2[0]) ** 2 + (goal_pose1[1] - current_pose2[1]) ** 2)
        if distance >= target_distance:
            goal_baselink = transform_pose_to_base_link(tf2[0], tf2[1], goal_pose1)
            goal_distance = distance
            goal_angle = math.atan2(goal_baselink[1],goal_baselink[0])
            if abs(goal_angle) < forward_goal_thresh:
                goals.append([goal_distance, goal_angle])
                lasers.append(laser2)
                if use_images:
                    images.append(image2)
                current_poses.append(current_pose2)
                goal_poses.append(goal_pose1)
                linear_velocities.append(linear_velocity2)
                angular_velocities.append(angular_velocity2)
                tfs.append(tf2)
        else:
            break

In [6]:
print('get motion commands')
motion_commands = np.concatenate([np.expand_dims(np.array(linear_velocities)[:,0], axis=1), np.expand_dims(np.array(linear_velocities)[:,1], axis=1), np.expand_dims(angular_velocities, axis=1)], axis=1)  # Shape: (1000, 3)

get motion commands


In [7]:
len(lasers)

227025

In [8]:
tf_file = os.path.join(input_dir, 'tfrecords/corr1_withImages_downsampled.tfrecord')

In [9]:

def serialize_metadata(dataset_length):
    feature = {
        'metadata': tf.train.Feature(bytes_list=tf.train.BytesList(value=[b"metadata"])),
        'length': tf.train.Feature(int64_list=tf.train.Int64List(value=[dataset_length]))
    }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()
    
def serialize_example(laser, goal, motion_command, image=None):
    if image is not None:
        feature = {
            'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[tf.io.serialize_tensor(image).numpy()])),
            # 'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[tf.io.encode_jpeg(image).numpy()])),
            'laser': tf.train.Feature(float_list=tf.train.FloatList(value=laser)),
            'goal': tf.train.Feature(float_list=tf.train.FloatList(value=goal)),
            'motion_command': tf.train.Feature(float_list=tf.train.FloatList(value=motion_command))
        }
    else:
        feature = {
            'laser': tf.train.Feature(float_list=tf.train.FloatList(value=laser)),
            'goal': tf.train.Feature(float_list=tf.train.FloatList(value=goal)),
            'motion_command': tf.train.Feature(float_list=tf.train.FloatList(value=motion_command))
        }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

def write_tfrecord(file_path, lasers, goal, motion_commands, images):
    dataset_length = len(lasers)
    # Create a list of indices and shuffle it
    indices = list(range(dataset_length))
    random.shuffle(indices)
    with tf.io.TFRecordWriter(file_path) as writer:
        writer.write(serialize_metadata(dataset_length))
        for i in indices:
            if len(images) !=0:
                example = serialize_example(
                    lasers[i], goal[i], motion_commands[i], images[i]
                )
            else:
                 example = serialize_example(
                lasers[i], goal[i], motion_commands[i]
            )
            writer.write(example)

write_tfrecord(tf_file, lasers, goals, motion_commands, images)

2024-10-31 02:00:49.576357: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [ ]:
sampled_lasers = []
sampled_images = []
sampled_goals = []
sampled_motion_commands = []
num_bins = 10  # Number of bins
samples_per_bin = 29000  # Number of samples per bin

x = np.array(goal)[:,1]
# Create bins based on 'x' values
bins = np.linspace(x.min(), x.max(), num_bins + 1)  # Create bin edges

# Digitize the 'x' values to assign them to bins
bin_indices = np.digitize(x, bins)



# Sample uniformly from each bin
for i in range(1, num_bins + 1):
    # Get indices of data points in the current bin
    indices_in_bin = np.where(bin_indices == i)[0]
    
    if len(indices_in_bin) > 0:
        # Randomly sample from the bin
        sampled_indices = np.random.choice(indices_in_bin, size=min(samples_per_bin, len(indices_in_bin)), replace=True)
        sampled_lasers.extend(np.array(lasers)[sampled_indices])
        if use_images:
            sampled_images.extend(np.array(images)[sampled_indices])
        sampled_goals.extend(np.array(goal)[sampled_indices])
        sampled_motion_commands.extend(np.array(motion_commands)[sampled_indices])

In [20]:
print(random.random())

0.05798887208369319


In [11]:
import tensorflow as tf
%load_ext tensorboard


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [12]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6013 (pid 286342), started 0:00:10 ago. (Use '!kill 286342' to kill it.)

In [4]:
!kill 84794